# Pregunta 3

A continuación se importan algunas librerías estándar de ``python`` que serán de utilidad. Luego se definen funciones auxiliares que serán usadas para manipular los textos planos y encriptados, así como la noción de distancia absoluta vista en clases (entre un string y la distribución de frecuencias de su alfabeto asociado):

In [91]:
# Standard library
from hashlib import md5, sha256

# Auxiliary functions

# Wrapper for MD5 hash function
def hash_md5(string):
    return md5(string.encode()).hexdigest()

# Wrapper for SHA-256 hash function
def hash_sha256(string):
    return sha256(string.encode()).hexdigest()

# Fake hash function that returns the same string (for testing)
def hash_fake(string):
    return string

A

In [92]:
# Merkle Tree representation
class MerkleTree:
    # Object representation for each node
    class Node:
        def __init__(self, value, level=0):
            self.value = value
            self.level = level
            self.parent = None
            self.sibling = None
            self.side = ''

    # Construct tree
    def __init__(self, strings, hash_func):
        """
        Arguments:
            strings: The set of strings S to be represented by the tree.
            hash_func: An arbitrary cryptographic hash function.
        """
        self.hash = hash_func
        self.leaves = [self.Node(self.hash(s)) for s in strings]

        # Bottom-Up tree generation using BFS
        nodes = [node for node in self.leaves]
        previous = None
        current = None
        level = 0
        while nodes:
            current = nodes.pop(0)
            if previous and current.level > level:  # Lonely node, self duplicate and create parent
                duplicated = MerkleTree.Node(previous.value, level=previous.level)
                parent = MerkleTree.Node(self.hash(previous.value + duplicated.value), level=previous.level + 1)
                previous.parent = parent
                previous.sibling = duplicated
                previous.side = 'i'
                duplicated.parent = parent
                duplicated.sibling = previous
                duplicated.side = 'd'
                nodes.append(parent)
                previous = current
            elif previous:  # Create parent of previous and current
                parent = MerkleTree.Node(self.hash(previous.value + current.value), level=current.level + 1)
                previous.parent = parent
                previous.sibling = current
                previous.side = 'i'
                current.parent = parent
                current.sibling = previous
                current.side = 'd'
                nodes.append(parent)
                previous = None
            else:  # Wait for next node to create a parent
                previous = current
            level = current.level  # Update current tree level

        # Special Case: The tree only represents one string
        if not level:
            duplicated = MerkleTree.Node(previous.value, level=previous.level)
            parent = MerkleTree.Node(self.hash(previous.value + duplicated.value), level=previous.level + 1)
            previous.parent = parent
            previous.sibling = duplicated
            previous.side = 'i'
            duplicated.parent = parent
            duplicated.sibling = previous
            duplicated.side = 'd'

    # TODO: Get root of the tree
    def get_root(self):
        # go to parent until there is no parent
        pass

    # TODO: Obtain necessary values to verify an item of the tree
    def get_proof_for(self, item):
        # get leaf, get sibling, go to parent, get sibling, go to parent, get sibling...repeat until root
        pass

    # Visual representation of the tree
    def __str__(self):
        repr_str = 'L0: '
        nodes = [node for node in self.leaves]
        previous = None
        current = None
        level = 0
        while nodes:
            current = nodes.pop(0)
            if current.level > level:
                level += 1
                repr_str += f'\nL{level}: '
                previous = None
            if previous:
                previous = None
            else:
                if current.parent:
                    repr_str += f'L({current.value}) R({current.sibling.value}) '
                    nodes.append(current.parent)
                else:
                    repr_str += f'ROOT({current.value})'
                previous = current
        return repr_str

In [93]:
# TODO: Verify proof for a given item and tree
def verify(root, item, proof, hash_func):
    """
    Arguments:
        root: The root of a Merkle Tree.
        item: An arbitrary string.
        proof: An alleged proof that item is part of a Merkle Tree with the previous root.
        hash_func: An arbitrary cryptographic hash function.
    Returns:
        correct: Whether the proof is correct or not.
    """
    # follow proof transformation of item until you get a possible root
    # compare possible root with root, if they are equal then its correct
    pass

A continuación 

In [94]:
# TODO: Test Merkle Tree
if __name__ == '__main__':
    # Strings to represent
    #strings = ['s1', 's2', 's3', 's4', 's5', 's6']
    strings = ['bitcoin', 'ethereum', 'tether', 'binance', 'usdc']
    #strings = ['']  # write a message in blocks

    # Merkle Tree
    tree = MerkleTree(strings, hash_fake)
    print(tree)

    # Verification
    #item = ''
    #verify(tree.get_root(), item, tree.get_proof_for(item), hash_fake)

L0: L(bitcoin) R(ethereum) L(tether) R(binance) L(usdc) R(usdc) 
L1: L(bitcoinethereum) R(tetherbinance) L(usdcusdc) R(usdcusdc) 
L2: L(bitcoinethereumtetherbinance) R(usdcusdcusdcusdc) 
L3: ROOT(bitcoinethereumtetherbinanceusdcusdcusdcusdc)
